# Mapping SOZ Contacts to Brain Regions

Purpose: Identify the brain regions associated with seizure onset zone (SOZ)
         contacts for a given subject using electrode pair data.

This script:
- Extracts SOZ contact labels 
- Matches those labels to bipolar electrode pairs in the dataset
- Retrieves the corresponding brain region annotations from multiple atlases
- Outputs a list of unique brain regions linked to the SOZ

Useful for: anatomical localization of seizure foci, 

Author: Noa Herz
Lab: Herz Lab | Jefferson University


## Step 1: imports and functions

In [131]:
# %reset
import pandas as pd
from cmlreaders import CMLReader, get_data_index

In [132]:
def getElecCats(reader):  
    try:
        elec_cats = reader.load('electrode_categories') # contains info about seizure onset zone, interictal spiking, broken leads.
        bad_elec_status=str(len(elec_cats))+' electrode categories'
    except:
        bad_elec_status= 'failed loading electrode categories'
        elec_cats = []
    return elec_cats,bad_elec_status       
    

In [133]:
def extract_pairs(reader):
    """
    Extracts bipolar channel pair information from a subject's dataset.
    
    In most cases, the pairs can be loaded directly using `reader.load('pairs')`. 
    However, for older experiments or non-standard formats (e.g., pyFR, FR1, catFR1),
    the data structure may vary, and the function attempts to reconstruct the pairs accordingly.
    
    Returns:
        pd.DataFrame: A dataframe containing contact_1, contact_2, label, and region information.
    """
    import pandas as pd
    import numpy as np
    from scipy.io import loadmat
    
    try:
        pairs = reader.load('pairs')
    except:
        if exp == 'pyFR':
            tal_path = f'/data/eeg/{sub}/tal/{sub}_talLocs_database_bipol.mat'
            try:
                df = pd.DataFrame(loadmat(tal_path)['bpTalStruct'][0])
            except:
                # Fallback for pyFR patients with a different field name
                df = pd.DataFrame(loadmat(tal_path)['subjTalEvents'][0])
            
            channel_info = [
                [
                    df['channel'][i][0][0],
                    df['channel'][i][0][1],
                    df['tagName'][i][0],
                    df['Loc5'][i][0]
                ] for i in range(len(df))
            ]
            pairs = pd.DataFrame(channel_info, columns=['contact_1', 'contact_2', 'label', 'stein.region'])

        elif exp in ['FR1', 'catFR1']:
            try:
                pairs = reader.load('pairs')
            except:
                from ptsa.data.readers import TalReader
                
                tal_path = f'/data/eeg/{sub}/tal/{sub}_talLocs_database_bipol.mat'
                tr = TalReader(tal_path)
                
                # Load bipolar pairs and Talairach location info
                _ = tr.get_bipolar_pairs()  # Not used directly here
                loc1 = tr.read()
                df = pd.DataFrame.from_records(loc1.tolist(), columns=np.array(loc1.dtype.names))
                
                channel_info = [
                    [
                        df['channel'][i][0],
                        df['channel'][i][1],
                        df['tagName'][i],
                        df['Loc5'][i]
                    ] for i in range(len(df))
                ]
                pairs = pd.DataFrame(channel_info, columns=['contact_1', 'contact_2', 'label', 'stein.region'])

    return pairs

## Step 2: Choose your data

In [134]:
# 1. See all subjects:
df = get_data_index("r1")
# df.iloc[10]

In [135]:
# random patient - I'm using row nomber 11 (index 10) as an example
sub = df.iloc[10]['subject']
exp = df.iloc[10]['experiment']
mont= df.iloc[10]['montage']
loc = df.iloc[10]['localization']
sess = df.iloc[10]['session']

reader = CMLReader(sub, exp, sess, montage=mont, localization=loc)


In [136]:
elec_cats=getElecCats(reader)
# See what's inside
elec_cats

({'soz': ['RDA2',
   'RDA3',
   'RDA4',
   'RMT1',
   'RMT2',
   'RMT3',
   'RMT4',
   'RPT1',
   'RPT2',
   'RPT3',
   'RPT4'],
  'interictal': ['LDA4',
   'LDA5',
   'LDH1',
   'LDH2',
   'LDH3',
   'RDA1',
   'RDA2',
   'RDA3'],
  'brain_lesion': ['NONE'],
  'bad_channel': ['LAD2']},
 '4 electrode categories')

# Step 3: find the labels of contacts within the SOZ

In [137]:
# Extract only the Seizure Onset Zone contacts
soz_list = elec_cats[0]['soz']
print(soz_list)

['RDA2', 'RDA3', 'RDA4', 'RMT1', 'RMT2', 'RMT3', 'RMT4', 'RPT1', 'RPT2', 'RPT3', 'RPT4']


# Step 4: locate the region corresponding to these contacts

In [138]:
# Extract all electrodes information:
pairs = extract_pairs(reader)
pd.set_option('display.max_columns', None)

pairs

,contact_1,contact_2,label,id,is_explicit,is_stim_only,type_1,type_2,avg.region,avg.x,avg.y,avg.z,avg.dural.region,avg.dural.x,avg.dural.y,avg.dural.z,dk.region,dk.x,dk.y,dk.z,ind.region,ind.x,ind.y,ind.z,ind.dural.region,ind.dural.x,ind.dural.y,ind.dural.z,tal.region,tal.x,tal.y,tal.z,stein.region,stein.x,stein.y,stein.z,das.region,das.x,das.y,das.z,mni.x,mni.y,mni.z
0,2,3,LAF1-LAF2,laf.1-laf.2,False,False,S,S,rostralmiddlefrontal,-41.205,52.315,4.545,rostralmiddlefrontal,-39.192639,50.946599,4.152569,rostralmiddlefrontal,NaN,NaN,NaN,rostralmiddlefrontal,-32.685,55.775,3.955,rostralmiddlefrontal,-32.235608,55.932797,4.261411,None,-41.03220,59.64390,3.984425,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-42.94905,56.40660,0.507948
1,3,4,LAF2-LAF3,laf.2-laf.3,False,False,S,S,parstriangularis,-47.010,42.985,2.890,rostralmiddlefrontal,-44.917132,42.021149,2.935475,rostralmiddlefrontal,NaN,NaN,NaN,rostralmiddlefrontal,-38.360,47.775,2.135,rostralmiddlefrontal,-37.853853,47.793965,2.376036,None,-47.39695,50.17595,2.985235,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-50.44460,47.53225,-0.108563
2,4,5,LAF3-LAF4,laf.3-laf.4,False,False,S,S,parstriangularis,-51.680,33.215,1.260,parstriangularis,-51.443475,32.776203,0.906133,parstriangularis,NaN,NaN,NaN,parstriangularis,-43.045,39.310,0.275,parstriangularis,-43.924270,40.200576,0.121013,None,-52.58225,40.20585,2.020270,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-55.93230,37.48010,1.210089
3,5,6,LAF4-LAF5,laf.4-laf.5,False,False,S,S,parstriangularis,-54.300,22.630,-0.890,parstriangularis,-53.127559,20.610749,1.078647,parstriangularis,NaN,NaN,NaN,parstriangularis,-45.965,30.025,-2.170,parstriangularis,-46.317560,30.780520,-2.236336,None,-55.65790,29.22995,0.533825,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-57.79805,27.55355,0.529655
4,6,7,LAF5-LAF6,laf.5-laf.6,False,False,S,S,superiortemporal,-56.380,12.085,-2.980,superiortemporal,-54.827944,9.408506,-1.535033,superiortemporal,NaN,NaN,NaN,parstriangularis,-48.395,20.745,-4.580,parstriangularis,-47.983551,20.135994,-5.284738,None,-58.16045,18.28200,-0.864941,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-58.72950,18.82430,-2.609848
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,90,91,RP3-RP4,rp.3-rp.4,False,False,S,S,postcentral,48.760,-31.595,54.150,postcentral,49.143819,-31.793900,53.966040,supramarginal,NaN,NaN,NaN,supramarginal,44.680,-27.855,40.820,supramarginal,44.980843,-27.691719,41.455778,None,53.18375,-22.11445,59.026000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,54.89260,-28.95750,51.072577
68,91,92,RP4-RP5,rp.4-rp.5,False,False,S,S,supramarginal,55.140,-30.045,45.440,supramarginal,57.377527,-30.900330,45.517476,supramarginal,NaN,NaN,NaN,supramarginal,50.145,-26.295,32.815,supramarginal,51.599941,-26.098420,34.332248,None,59.58060,-22.20950,49.813200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,58.75835,-26.06735,43.368032
69,92,93,RP5-RP6,rp.5-rp.6,False,False,S,S,supramarginal,59.990,-28.425,35.470,supramarginal,62.212215,-29.187280,35.959248,supramarginal,NaN,NaN,NaN,supramarginal,54.220,-24.515,23.735,supramarginal,55.418203,-24.499221,25.150691,None,64.33270,-22.35900,39.325900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,63.29850,-23.67250,31.908984
70,93,94,RP6-RP7,rp.6-rp.7,False,False,S,S,supramarginal,62.765,-27.405,24.635,supramarginal,61.760509,-26.655133,25.863125,NaN,NaN,NaN,NaN,supramarginal,56.395,-23.115,13.920,supramarginal,56.266446,-23.261644,14.763449,None,66.86295,-23.15485,28.022650,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,66.20690,-23.55400,19.837947


### Now, let's find out which brain regions corresponds to the SOZ contacts.

1. Match each contact in soz_list (e.g., 'RDA2', 'RMT1') to the corresponding label in the pairs DataFrame (e.g., 'RDA2-RDA3', 'RMT1-RMT2').

2. Identify rows where either contact name appears as part of the label.

3. Extract the associated brain region, such as from the avg.region column.



In [139]:
# Initialize an empty list to hold True/False for each row
soz_mask = []

# Loop through each row in the DataFrame
for label in pairs['label']:
    found = False
    for soz in soz_list:
        if soz in label:
            found = True
            break  # No need to keep checking once we've found a match
    soz_mask.append(found)

# Convert the list to a boolean mask and apply to the DataFrame
matched_soz_pairs = pairs[soz_mask]

# Display the matched rows.
# These columns reflect brain region assignments from different atlases
print(matched_soz_pairs[['label', 'avg.region','stein.region','ind.region','dk.region']])

        label avg.region stein.region        ind.region         dk.region
51  RMT1-RMT2    lingual          NaN           lingual           lingual
52  RMT2-RMT3   fusiform          NaN          fusiform          fusiform
53  RMT3-RMT4   fusiform          NaN          fusiform          fusiform
54  RPT1-RPT2   fusiform          NaN  lateraloccipital  lateraloccipital
55  RPT2-RPT3   fusiform          NaN          fusiform          fusiform
56  RPT3-RPT4   fusiform          NaN          fusiform          fusiform
62  RDA1-RDA2       None     Right EC              None               NaN
63  RDA2-RDA3       None    Right CA1              None               NaN
64  RDA3-RDA4       None          NaN              None               NaN


In [140]:
# You can ensure that the list is accurate by reviewing the SOZ again
print(soz_list)

['RDA2', 'RDA3', 'RDA4', 'RMT1', 'RMT2', 'RMT3', 'RMT4', 'RPT1', 'RPT2', 'RPT3', 'RPT4']


### Define which atlases you want to use. 
The Stein atlas may contain detailed anatomy of the hippocampus, and is therefore the most accurate if present. Here, I am concatanating the avg.region with the stein atlas to obtain full anatomical localization

regions_list = matched_soz_pairs['avg.region'].tolist() + matched_soz_pairs['stein.region'].tolist()

In [148]:
# Remove None and NaN values, and return only unique region names
# 'x == x' is a simple trick to filter out NaNs, which are not equal to themselves
soz_regions = sorted(set(filter(lambda x: x is not None and x == x, regions_list)))

# Print the final list of unique brain regions associated with SOZ contacts
print("\nUnique brain regions associated with SOZ contacts:")
for region in soz_regions:
    print(f"- {region}")


Unique brain regions associated with SOZ contacts:
- Right CA1
- Right EC
- fusiform
- lingual
